# Load Data and Final Clean

In [1]:
df <- read.csv(file="data_010820.csv",sep=",",header=TRUE)

In [3]:
colnames(df)
dim(df)

[1] "X"                          "X1st.Term.Funding"         
 [3] "X2nd.Term.Funding"          "X3rd.Term.Funding"         
 [5] "X4th.Term.Funding"          "Start.age"                 
 [7] "College"                    "Current.Age"               
 [9] "Degree"                     "Gender"                    
[11] "Grad.Funding.Indicator"     "ID"                        
[13] "Citizenship"                "Major.ID"                  
[15] "Major"                      "Net.Cost.After.Any.Funding"
[17] "Program"                    "Prep.Level"                
[19] "Total.Charges"              "Total.Funding.Amount"      
[21] "Y"                          "PC1"                       
[23] "PC2"                        "PC3"                       
[25] "Term.1.GPA"                 "Term.2.GPA"                
[27] "Term.3.GPA"                 "Term.4.GPA"                
[29] "Term.5.GPA"                 "Term.6.GPA"                
[31] "Term.7.GPA"                 "Term.8.GPA"                
[33] "Gap"                        "Slope"                     
[35] "Valid.term"                 "Average"                   
[37] "Academic.Period..Years."

[1] 2470   37

In [4]:
colnames(df)[16] = 'net_cost'
colnames(df)[34] = 'GPA_trend'
colnames(df)[37] = 'academic_period'
df$net_cost = df$net_cost/10000

## Seperate by School

Different Schools

In [5]:
levels(factor(df$College))

[1] "GD CCPA"        "GD Education"   "GD Grad School" "GD Harpur"     
[5] "GD Management"  "GD Nursing"     "GD Watson"

In [11]:
#Harpuer_df = df[which(df$College %in% c("GD Harpur, GD Education, GD Grad School","GD CCPA")),]
Harpuer_df = df[which(df$College %in% c("GD Harpur")),]
Management_df = df[which(df$College=="GD Management"),]
Nursing_df = df[which(df$College=="GD Nursing"),]
Watson_df = df[which(df$College=="GD Watson"),]

In [24]:
levels(Management_df$Major)

[1] "Anthropology"                          
 [2] "Art History"                           
 [3] "Art History Combined MA PhD"           
 [4] "Behavioral Neuroscience"               
 [5] "Biological Sciences"                   
 [6] "Biological Sciences EEB"               
 [7] "Biomedical Engineering"                
 [8] "Chemistry"                             
 [9] "Clinical Psychology"                   
[10] "Cognitive Psychology"                  
[11] "Community and Public Affairs"          
[12] "Community Health NP"                   
[13] "Comparative Literature"                
[14] "Computer Science"                      
[15] "Economics"                             
[16] "Educational Theory and Practic"        
[17] "Electrical and Computer Engr"          
[18] "Electrical Engineering"                
[19] "English"                               
[20] "Family Nurse Practitioner"             
[21] "Family Psych Mental Health NP"         
[22] "Geology"                               
[23] "History"                               
[24] "Industrial and Systems Enginee"        
[25] "Management PHD"                        
[26] "Material Sci and Eng-Engineerg"        
[27] "Materials Engineering"                 
[28] "Materials Science and Engineer"        
[29] "Materials Science and Engr"            
[30] "Mathematics"                           
[31] "Mathematics/Economics"                 
[32] "Mechanical Engineering"                
[33] "Nursing"                               
[34] "Philosophy Interpretation and"         
[35] "Philosophy SPEL"                       
[36] "Physics"                               
[37] "Political Science"                     
[38] "Sociology"                             
[39] "Systems Science"                       
[40] "Systems Science/Biomedical Engineering"
[41] "Systems Science/EE"                    
[42] "Translation Studies"                   
[43] "X Geology"                             
[44] "X Gerontological NP"                   
[45] "XGerontolog Clin Nurse Special"

In [72]:
a_n = dim(Harpuer_df)[1]
a_1 = sum(Harpuer_df$Y)
a_0 = a_n - a_1
b_n = dim(CCPA_df)[1]
b_1 = sum(CCPA_df$Y)
b_0 = b_n - b_1
c_n = dim(Education_df)[1]
c_1 = sum(Education_df$Y)
c_0 = c_n - c_1
d_n = dim(Grad_df)[1]
d_1 = sum(Grad_df$Y)
d_0 = d_n - d_1
e_n = dim(Management_df)[1]
e_1 = sum(Management_df$Y)
e_0 = e_n - e_1
f_n = dim(Nursing_df)[1]
f_1 = sum(Nursing_df$Y)
f_0 = f_n - f_1
g_n = dim(Watson_df)[1]
g_1 = sum(Watson_df$Y)
g_0 = g_n - g_1
School = c("Harpur","CCPA","Education","Grad","SOM","Nursing","Watson")
sch_total = c(a_n,b_n,c_n,d_n,e_n,f_n,g_n)
succ_total = c(a_1,b_1,c_1,d_1,e_1,f_1,g_1)
unsucc_total = c(a_0,b_0,c_0,d_0,e_0,f_0,g_0)
data.frame("School"=School,"Total"=sch_total,"Successful Graduates"=succ_total,"Unsuccessful Graduates"=unsucc_total)

School,Total,Successful.Graduates,Unsuccessful.Graduates
Harpur,1606,961,645
CCPA,26,12,14
Education,75,42,33
Grad,73,62,11
SOM,58,38,20
Nursing,117,72,45
Watson,515,343,172


# Harpur Analysis

Won't use Degree column since it does not accurately show the student's major

<b> Logistic regression model:

In [73]:
log_mod_h = glm(Y~Start.age+factor(Gender)+factor(Grad.Funding.Indicator)+Citizenship+
                net_cost+PC1+PC2+PC3+factor(Gap)+GPA_trend+academic_period,
                data=Harpuer_df,family="binomial")
summary(log_mod_h)


Call:
glm(formula = Y ~ Start.age + factor(Gender) + factor(Grad.Funding.Indicator) + 
    Citizenship + net_cost + PC1 + PC2 + PC3 + factor(Gap) + 
    GPA_trend + academic_period, family = "binomial", data = Harpuer_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-3.0966  -0.9420   0.5913   0.8977   2.0387  

Coefficients:
                                              Estimate Std. Error z value
(Intercept)                                  -1.076450   0.509903  -2.111
Start.age                                     0.002942   0.009686   0.304
factor(Gender)Male                            0.024552   0.114791   0.214
factor(Grad.Funding.Indicator)1               1.005712   0.153767   6.540
CitizenshipAmerican Indian or Alaskan Native -1.929077   0.919019  -2.099
CitizenshipAsia                              -0.058179   0.383520  -0.152
CitizenshipAsian                             -1.178367   0.465538  -2.531
CitizenshipBlack or African American         -0.526495  

<b> Significant Variables of logistic regression model: 

In [74]:
round(summary(log_mod_h)$coeff[which(summary(log_mod_h)$coef[,4] <= .05),c(1,4)],5)

,Estimate,Pr(>|z|)
(Intercept),-1.07645,0.03476
factor(Grad.Funding.Indicator)1,1.00571,0.00000
CitizenshipAmerican Indian or Alaskan Native,-1.92908,0.03581
CitizenshipAsian,-1.17837,0.01137
PC1,-0.07047,0.01168
PC2,-0.26241,0.00000
academic_period,0.22268,0.00000


Things we need
-  Run AIC/BIC stepwise model selection for Harpur and summarize results
-  Repeat for the other schools

<b> Backward AIC:

In [75]:
library('MASS')
log_mod_h_aic = stepAIC(log_mod_h, direction = 'backward')

Start:  AIC=1875.28
Y ~ Start.age + factor(Gender) + factor(Grad.Funding.Indicator) + 
    Citizenship + net_cost + PC1 + PC2 + PC3 + factor(Gap) + 
    GPA_trend + academic_period

                                 Df Deviance    AIC
- net_cost                        1   1829.3 1873.3
- factor(Gender)                  1   1829.3 1873.3
- Start.age                       1   1829.4 1873.4
- Citizenship                    12   1851.4 1873.4
- GPA_trend                       1   1831.0 1875.0
- factor(Gap)                     1   1831.2 1875.2
<none>                                1829.3 1875.3
- PC3                             1   1832.7 1876.7
- PC1                             1   1835.7 1879.7
- PC2                             1   1872.0 1916.0
- factor(Grad.Funding.Indicator)  1   1874.1 1918.1
- academic_period                 1   1949.4 1993.4

Step:  AIC=1873.3
Y ~ Start.age + factor(Gender) + factor(Grad.Funding.Indicator) + 
    Citizenship + PC1 + PC2 + PC3 + factor(Gap) + GPA_tr

<b> Significant variables backward BIC selects:

In [76]:
round(summary(log_mod_h_aic)$coeff[which(summary(log_mod_h_aic)$coef[,4] <= .05),c(1,4)],5)

,Estimate,Pr(>|z|)
(Intercept),-1.36412,0.00000
factor(Grad.Funding.Indicator)1,0.99276,0.00000
PC1,-0.06858,0.01239
PC2,-0.26974,0.00000
academic_period,0.22377,0.00000


<b> Backward BIC

In [77]:
log_mod_h_bic = stepAIC(log_mod_h, direction = "backward", k = log(a_n))

Start:  AIC=1999.05
Y ~ Start.age + factor(Gender) + factor(Grad.Funding.Indicator) + 
    Citizenship + net_cost + PC1 + PC2 + PC3 + factor(Gap) + 
    GPA_trend + academic_period

                                 Df Deviance    AIC
- Citizenship                    12   1851.4 1932.6
- net_cost                        1   1829.3 1991.7
- factor(Gender)                  1   1829.3 1991.7
- Start.age                       1   1829.4 1991.8
- GPA_trend                       1   1831.0 1993.4
- factor(Gap)                     1   1831.2 1993.6
- PC3                             1   1832.7 1995.0
- PC1                             1   1835.7 1998.1
<none>                                1829.3 1999.0
- PC2                             1   1872.0 2034.4
- factor(Grad.Funding.Indicator)  1   1874.1 2036.5
- academic_period                 1   1949.4 2111.8

Step:  AIC=1932.63
Y ~ Start.age + factor(Gender) + factor(Grad.Funding.Indicator) + 
    net_cost + PC1 + PC2 + PC3 + factor(Gap) + GPA_tren

<b> Significant variables backward BIC selects:

In [66]:
round(summary(log_mod_h_bic)$coeff[which(summary(log_mod_h_bic)$coef[,4] <= .05),c(1,4)],5)

,Estimate,Pr(>|z|)
(Intercept),-1.42580,0
factor(Grad.Funding.Indicator)1,1.01422,0
PC2,-0.24977,0
academic_period,0.22815,0


# Management Analysis

In [11]:
log_mod_m = glm(Y~Start.age+factor(Gender)+
                Net.Cost.After.Any.Funding+PC1+PC2+PC3+factor(Gap)+Slope,
                data=Management_df,family="binomial")
summary(log_mod_m)


Call:
glm(formula = Y ~ Start.age + factor(Gender) + Net.Cost.After.Any.Funding + 
    PC1 + PC2 + PC3 + factor(Gap) + Slope, family = "binomial", 
    data = Management_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.0067  -0.5098   0.1893   0.6294   1.8714  

Coefficients:
                             Estimate Std. Error z value Pr(>|z|)   
(Intercept)                 8.763e-01  2.516e+00   0.348   0.7277   
Start.age                  -1.136e-01  9.137e-02  -1.243   0.2139   
factor(Gender)Male         -1.295e+00  8.219e-01  -1.575   0.1152   
Net.Cost.After.Any.Funding  2.000e-04  7.188e-05   2.782   0.0054 **
PC1                        -1.171e+00  1.097e+00  -1.067   0.2859   
PC2                        -1.352e+00  1.082e+00  -1.250   0.2114   
PC3                         4.252e-02  8.557e-02   0.497   0.6193   
factor(Gap)1                1.568e+01  1.748e+03   0.009   0.9928   
Slope                       1.621e+01  8.749e+00   1.852   0.0640 . 
---
Si

In [12]:
round(summary(log_mod_m)$coeff[,c(1,4)],5)

,Estimate,Pr(>|z|)
(Intercept),0.87631,0.72766
Start.age,-0.11355,0.21395
factor(Gender)Male,-1.29453,0.11525
Net.Cost.After.Any.Funding,0.00020,0.00540
PC1,-1.17073,0.28591
PC2,-1.35173,0.21140
PC3,0.04252,0.61925
factor(Gap)1,15.67937,0.99284
Slope,16.20614,0.06397


<b> Significant Variables: Net cost, slope.

In [13]:
log_mod_m_aic_b = stepAIC(log_mod_m, direction = 'backward')

Start:  AIC=64.12
Y ~ Start.age + factor(Gender) + Net.Cost.After.Any.Funding + 
    PC1 + PC2 + PC3 + factor(Gap) + Slope

                             Df Deviance    AIC
- PC3                         1   46.370 62.370
- factor(Gap)                 1   47.193 63.193
- Start.age                   1   47.735 63.735
<none>                            46.121 64.121
- PC1                         1   48.812 64.812
- factor(Gender)              1   48.865 64.865
- PC2                         1   50.095 66.095
- Slope                       1   50.894 66.894
- Net.Cost.After.Any.Funding  1   58.239 74.239

Step:  AIC=62.37
Y ~ Start.age + factor(Gender) + Net.Cost.After.Any.Funding + 
    PC1 + PC2 + factor(Gap) + Slope

                             Df Deviance    AIC
- factor(Gap)                 1   47.351 61.351
- Start.age                   1   47.829 61.829
<none>                            46.370 62.370
- factor(Gender)              1   48.986 62.986
- PC1                         1   49.2

<b> Variables backward AIC selects: Net cost, PC1, PC2, slope, gender.

In [14]:
log_mod_m_f = glm(Y ~ factor(Gender) + Net.Cost.After.Any.Funding + PC1 + PC2 + Slope,
                 data=Management_df,family="binomial")
summary(log_mod_m_f)


Call:
glm(formula = Y ~ factor(Gender) + Net.Cost.After.Any.Funding + 
    PC1 + PC2 + Slope, family = "binomial", data = Management_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3779  -0.6118   0.2405   0.6805   1.4678  

Coefficients:
                             Estimate Std. Error z value Pr(>|z|)   
(Intercept)                -2.120e+00  1.248e+00  -1.699  0.08936 . 
factor(Gender)Male         -1.236e+00  8.128e-01  -1.520  0.12846   
Net.Cost.After.Any.Funding  1.916e-04  6.509e-05   2.944  0.00324 **
PC1                        -1.275e+00  1.004e+00  -1.270  0.20416   
PC2                        -1.444e+00  9.706e-01  -1.488  0.13672   
Slope                       1.742e+01  9.092e+00   1.916  0.05531 . 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 74.726  on 57  degrees of freedom
Residual deviance: 48.982  on 52  degrees of freedom
AIC: 60.982

Numbe

In [15]:
round(summary(log_mod_m_f)$coeff[,c(1,4)],5)

,Estimate,Pr(>|z|)
(Intercept),-2.12045,0.08936
factor(Gender)Male,-1.23559,0.12846
Net.Cost.After.Any.Funding,0.00019,0.00324
PC1,-1.27537,0.20416
PC2,-1.44430,0.13672
Slope,17.42356,0.05531


<b> Significant Variables after backward AIC selection: Net cost.

# Nursing Analysis

In [16]:
log_mod_n = glm(Y~Start.age+factor(Gender)+factor(Grad.Funding.Indicator)+
                Net.Cost.After.Any.Funding+PC1+PC2+PC3+factor(Gap)+Slope,
                data=Nursing_df,family="binomial")
summary(log_mod_n)


Call:
glm(formula = Y ~ Start.age + factor(Gender) + factor(Grad.Funding.Indicator) + 
    Net.Cost.After.Any.Funding + PC1 + PC2 + PC3 + factor(Gap) + 
    Slope, family = "binomial", data = Nursing_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.5244  -1.0121   0.3747   0.9610   1.4857  

Coefficients:
                                  Estimate Std. Error z value Pr(>|z|)   
(Intercept)                      6.437e+00  4.889e+02   0.013  0.98950   
Start.age                       -2.114e-04  1.962e-02  -0.011  0.99140   
factor(Gender)Male              -2.864e-01  6.295e-01  -0.455  0.64912   
factor(Grad.Funding.Indicator)1  6.651e-01  4.773e-01   1.393  0.16348   
Net.Cost.After.Any.Funding       4.082e-05  1.501e-05   2.720  0.00654 **
PC1                             -5.019e-01  4.325e+02  -0.001  0.99907   
PC2                              6.240e+00  4.280e+02   0.015  0.98837   
PC3                             -1.313e+01  7.327e+02  -0.018  0.98570   


In [17]:
round(summary(log_mod_n)$coeff[,c(1,4)],5)

,Estimate,Pr(>|z|)
(Intercept),6.43660,0.98950
Start.age,-0.00021,0.99140
factor(Gender)Male,-0.28641,0.64912
factor(Grad.Funding.Indicator)1,0.66514,0.16348
Net.Cost.After.Any.Funding,0.00004,0.00654
PC1,-0.50195,0.99907
PC2,6.23961,0.98837
PC3,-13.12930,0.98570
factor(Gap)1,-0.61335,0.51621
Slope,-2.20414,0.66939


<b> Significant Variables: Net cost.

In [18]:
log_mod_n_aic_b = stepAIC(log_mod_n, direction = 'backward')

Start:  AIC=146.2
Y ~ Start.age + factor(Gender) + factor(Grad.Funding.Indicator) + 
    Net.Cost.After.Any.Funding + PC1 + PC2 + PC3 + factor(Gap) + 
    Slope

                                 Df Deviance    AIC
- PC1                             1   126.20 144.20
- Start.age                       1   126.20 144.20
- Slope                           1   126.38 144.38
- factor(Gender)                  1   126.41 144.41
- factor(Gap)                     1   126.62 144.62
- factor(Grad.Funding.Indicator)  1   128.19 146.19
<none>                                126.20 146.20
- PC2                             1   135.10 153.10
- Net.Cost.After.Any.Funding      1   135.56 153.56
- PC3                             1   138.87 156.87

Step:  AIC=144.2
Y ~ Start.age + factor(Gender) + factor(Grad.Funding.Indicator) + 
    Net.Cost.After.Any.Funding + PC2 + PC3 + factor(Gap) + Slope

                                 Df Deviance    AIC
- Start.age                       1   126.20 142.20
- Slope    

<b> Variables backward AIC selects: Net cost, PC2, PC3, funding indicator.

In [19]:
log_mod_n_f = glm(Y ~ factor(Gender) + Net.Cost.After.Any.Funding + PC2 + PC3 + factor(Grad.Funding.Indicator),
                 data=Nursing_df,family="binomial")
summary(log_mod_n_f)


Call:
glm(formula = Y ~ factor(Gender) + Net.Cost.After.Any.Funding + 
    PC2 + PC3 + factor(Grad.Funding.Indicator), family = "binomial", 
    data = Nursing_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.5308  -0.9955   0.3976   0.9511   1.5144  

Coefficients:
                                  Estimate Std. Error z value Pr(>|z|)   
(Intercept)                      6.519e+00  4.692e+02   0.014  0.98892   
factor(Gender)Male              -2.396e-01  6.161e-01  -0.389  0.69738   
Net.Cost.After.Any.Funding       3.868e-05  1.444e-05   2.679  0.00738 **
PC2                              6.398e+00  4.222e+02   0.015  0.98791   
PC3                             -1.345e+01  7.294e+02  -0.018  0.98529   
factor(Grad.Funding.Indicator)1  6.746e-01  4.507e-01   1.497  0.13444   
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 155.91  on 116  degrees of freedom
Residual

In [20]:
round(summary(log_mod_n_f)$coeff[,c(1,4)],5)

,Estimate,Pr(>|z|)
(Intercept),6.51917,0.98892
factor(Gender)Male,-0.23956,0.69738
Net.Cost.After.Any.Funding,0.00004,0.00738
PC2,6.39775,0.98791
PC3,-13.44851,0.98529
factor(Grad.Funding.Indicator)1,0.67461,0.13444


<b> Significant Variables after backward AIC selection: Net cost.

# Watson Analysis 

In [21]:
log_mod_w = glm(Y~Start.age+factor(Gender)+factor(Grad.Funding.Indicator)+Citizenship+
                Net.Cost.After.Any.Funding+PC1+PC2+PC3+factor(Gap)+Slope,
                data=Watson_df,family="binomial")
summary(log_mod_w)


Call:
glm(formula = Y ~ Start.age + factor(Gender) + factor(Grad.Funding.Indicator) + 
    Citizenship + Net.Cost.After.Any.Funding + PC1 + PC2 + PC3 + 
    factor(Gap) + Slope, family = "binomial", data = Watson_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.6172  -0.9174   0.4931   0.8008   1.7480  

Coefficients:
                                       Estimate Std. Error z value Pr(>|z|)    
(Intercept)                           1.317e+01  1.455e+03   0.009  0.99278    
Start.age                            -1.843e-02  1.792e-02  -1.029  0.30370    
factor(Gender)Male                    3.620e-01  2.729e-01   1.327  0.18467    
factor(Grad.Funding.Indicator)1       1.639e+00  2.817e-01   5.820 5.89e-09 ***
CitizenshipAfrica                     6.043e-01  1.572e+03   0.000  0.99969    
CitizenshipAsia                      -1.372e+01  1.455e+03  -0.009  0.99248    
CitizenshipAsian                     -1.446e+01  1.455e+03  -0.010  0.99207    
CitizenshipBl

In [22]:
round(summary(log_mod_w)$coeff[,c(1,4)],5)

,Estimate,Pr(>|z|)
(Intercept),13.17397,0.99278
Start.age,-0.01843,0.30370
factor(Gender)Male,0.36203,0.18467
factor(Grad.Funding.Indicator)1,1.63919,0.00000
CitizenshipAfrica,0.60430,0.99969
CitizenshipAsia,-13.72175,0.99248
CitizenshipAsian,-14.46061,0.99207
CitizenshipBlack or African American,-14.54767,0.99202
CitizenshipChina,-14.26875,0.99218
CitizenshipEurope,-14.65364,0.99197


<b> Significant Variables: Funding indicator, net cost, PC1, PC2, gap.

In [23]:
log_mod_w_aic_b = stepAIC(log_mod_w, direction = 'backward')

Start:  AIC=572.53
Y ~ Start.age + factor(Gender) + factor(Grad.Funding.Indicator) + 
    Citizenship + Net.Cost.After.Any.Funding + PC1 + PC2 + PC3 + 
    factor(Gap) + Slope

                                 Df Deviance    AIC
- Citizenship                    12   544.67 564.67
- Start.age                       1   529.60 571.60
- factor(Gender)                  1   530.27 572.27
<none>                                528.53 572.53
- Slope                           1   531.22 573.22
- PC3                             1   531.70 573.70
- PC2                             1   534.39 576.39
- factor(Gap)                     1   538.67 580.67
- Net.Cost.After.Any.Funding      1   547.78 589.78
- PC1                             1   549.04 591.04
- factor(Grad.Funding.Indicator)  1   564.72 606.72

Step:  AIC=564.67
Y ~ Start.age + factor(Gender) + factor(Grad.Funding.Indicator) + 
    Net.Cost.After.Any.Funding + PC1 + PC2 + PC3 + factor(Gap) + 
    Slope

                                 Df 

<b>Variables backward AIC selects: Funding indicator, net cost, PC1, PC2, PC3, gap, slope.

In [24]:
log_mod_w_f = glm(Y ~ factor(Grad.Funding.Indicator) + Net.Cost.After.Any.Funding + PC1 + PC2 + PC3 + factor(Gap) + Slope,
                 data=Watson_df,family="binomial")
summary(log_mod_w_f)


Call:
glm(formula = Y ~ factor(Grad.Funding.Indicator) + Net.Cost.After.Any.Funding + 
    PC1 + PC2 + PC3 + factor(Gap) + Slope, family = "binomial", 
    data = Watson_df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.7433  -0.9562   0.5226   0.7878   1.6898  

Coefficients:
                                  Estimate Std. Error z value Pr(>|z|)    
(Intercept)                     -1.379e+00  2.713e-01  -5.083 3.71e-07 ***
factor(Grad.Funding.Indicator)1  1.747e+00  2.564e-01   6.812 9.65e-12 ***
Net.Cost.After.Any.Funding       4.348e-05  9.976e-06   4.358 1.31e-05 ***
PC1                             -2.521e-01  5.984e-02  -4.213 2.52e-05 ***
PC2                             -1.640e-01  8.056e-02  -2.035  0.04181 *  
PC3                             -1.060e+00  5.008e-01  -2.116  0.03437 *  
factor(Gap)1                     1.001e+00  3.692e-01   2.711  0.00671 ** 
Slope                            1.653e+00  1.122e+00   1.473  0.14075    
---
Signif. codes:  0

In [25]:
round(summary(log_mod_w_f)$coeff[,c(1,4)],5)

,Estimate,Pr(>|z|)
(Intercept),-1.37886,0.00000
factor(Grad.Funding.Indicator)1,1.74657,0.00000
Net.Cost.After.Any.Funding,0.00004,0.00001
PC1,-0.25209,0.00003
PC2,-0.16398,0.04181
PC3,-1.05955,0.03437
factor(Gap)1,1.00082,0.00671
Slope,1.65303,0.14075


<b> Significant Variables after backward AIC selection: Funding indicator, net cost, PC1, PC2, PC3, gap.